In [6]:
from bs4 import BeautifulSoup
import time
import os
import urllib.request
from urllib.error import URLError, HTTPError

from selenium import webdriver
import re
import pandas as pd

In [7]:
path = "C:/chromedriver.exe" # chromedriver.exe 다운로드 필수
driver = webdriver.Chrome(path)

In [8]:
url1='https://www.youthcenter.go.kr/jynEmpSptNew/jynEmpSptList.do?pageIndex=1&currentPage='
url2='&sortField=exprRnkn&setZzimBusiId=&bizId=&plcySel=&chargerOrgCdSel=&sortField2=exprRnkn&higdBlowAccrYnSel=&hiscEnrlYnSel=&hiscGrdnPrerYnSel=&higdAccrYnSel=&univEnrlYnSel=&univGrdnAccrYnSel=&drAccrYnSel=&majrSglSersYnSel=&majrSocySersYnSel=&majrBsflSersYnSel=&majrNtssSersYnSel=&majrEngrSersYnSel=&majrEnspSersYnSel=&splzRlmSmpzEmpmYnSel=&splzRlmFmleYnSel=&splzRlmLigYnSel=&splzRlmDspnYnSel=&splzRlmSdrYnSel=&splzRlmFrmlYnSel=&empnHffcYnSel=&empyBsmgYnSel=&empmUnpnYnSel=&scroll=1&btnSearchTxt=&srchAge=&higdBlowAccrYn=on&hiscEnrlYn=on&hiscGrdnPrerYn=on&higdAccrYn=on&univEnrlYn=on&univGrdnAccrYn=on&drAccrYn=on&majrSglSersYn=on&majrSocySersYn=on&majrBsflSersYn=on&majrNtssSersYn=on&majrEngrSersYn=on&majrEnspSersYn=on&splzRlmSmpzEmpmYn=on&splzRlmFmleYn=on&splzRlmLigYn=on&splzRlmDspnYn=on&splzRlmSdrYn=on&splzRlmFrmlYn=on&empnHffcYn=on&empyBsmgYn=on&empmUnpnYn=on'
page_num=1

In [9]:
regex_digit = re.compile("\'\d+\'")

In [10]:
regex_digit2 = re.compile("\d+")

while로 교체필요

In [11]:
def id_crawl():
    
    res_list=[]
    page_num=1
    while(1):
        driver.get(url1 + str(page_num)+url2)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        res_ids = regex_digit.findall(str(soup.select('#jynEmpSptList1')))
        
        if len(res_ids)==0:
            break;
            
        res_list.extend(res_ids)
        page_num = page_num + 1
        
    return res_list

In [12]:
id_list = id_crawl()

In [13]:
def del_up_dot(ids_list):
    res = []
    for i in ids_list:
        x=regex_digit2.findall(i)
        res.extend(x)
        
    return res

In [14]:
id_list2 = del_up_dot(id_list)

In [15]:
url_id_base = "https://www.youthcenter.go.kr/jynEmpSptNew/jynEmpSptGuide.do?bizId="

In [16]:
col_list = ['id','title','hash tag','설명','연령','학력','전공','취업상태','특화분야','기업','신청기간','기타내용']

In [17]:
pattern = re.compile(r'[^\t\n]+')

In [18]:
info_table=pd.DataFrame(columns=col_list)

In [19]:
def chung_crawl(ids_list):
    num_df = 0
    col_list = ['id','title','hash tag','설명','연령','학력','전공','취업상태','특화분야','기업','신청기간','기타내용']
    index_list = ['연령','학력','전공','취업상태','특화분야','기업','신청기간','기타내용']
    info_table=pd.DataFrame(columns=col_list)
    for i in ids_list:
        driver.get(url_id_base+str(i))
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
    
        sample_list=[]
        exp=[]
        exp.append(i)
        idx=soup.select('h3.doc_tit02')[0].text
        string1 = pattern.findall(str(idx))
        exp.append(string1[0])
        if len(string1)>=3:
            exp.append(string1[2])
        else:
            exp.append('-')
    
    
        idx=soup.select('h4.bullet-arrow1')[0].text
        string2=pattern.findall(str(idx))
        if len(string2)!=0:
            exp.append(string2[0])
        else:
            exp.append('-')
  
    
        idx=soup.select('ul.common_table02')[0].text
        string=pattern.findall(str(idx))

        remove_space = list(filter(lambda x: x!=' ', string))
    
        for kda in index_list:
            num=remove_space.index(kda)
            exp.append(remove_space[num+1])
    

        info_table.loc[num_df]=exp
        num_df = num_df+1
    return info_table

In [20]:
df = chung_crawl(id_list2)

In [47]:
df['id'] = df['id'].apply(lambda x: url_id_base+str(x))

In [51]:
df.loc[1]['id']

'https://www.youthcenter.go.kr/jynEmpSptNew/jynEmpSptGuide.do?bizId=201808230013'

In [22]:
df.columns

Index(['id', 'title', 'hash tag', '설명', '연령', '학력', '전공', '취업상태', '특화분야', '기업',
       '신청기간', '기타내용'],
      dtype='object')

In [21]:
df

,id,title,hash tag,설명,연령,학력,전공,취업상태,특화분야,기업,신청기간,기타내용
0,201808160009,국가근로장학금,학자금 지원,안정적인 학업여건 조성과 취업역량 제고를 위한 장학금,무관,"국내 대학의 재학생(입학예정자 포함),",무관,무관,-,-,공고 확인 후 신청,-
1,201808230013,"월드프렌즈(World Friends Korea, WFK) - 일반봉사단",해외진출,개발도상국에 대한 주요 무상원조사업의 하나로 개발도상국의 수요를 바탕으로 우리가 비...,"만 19세 이상(단, 남자는 군필이거나 면제)",무관,무관,무관(기업체 등에 소속된 지원자는 소속기관과 협의하여 국내교육 입교 전까지 신분 정...,-,-,"연 8회 모집, 홈페이지 참조",-
2,201903140005,청년구직활동지원금,"교육훈련·체험·인턴,",스스로 취업을 준비하는 청년에게 월 50만원씩 최대 6개월 간 지원,만 18~34세,고등학교‧대학교‧대학원 졸업 또는 중퇴 후 2년 이내,무관,"미취업(단, 주 근로시간 20시간 이하인 경우 미취업으로 간주)",무관,-,매월 1일~20일,가구소득: 기준 중위소득 120% 이하
3,201902270001,청년취업성공패키지,"교육훈련·체험·인턴,",단계별로 차근차근 취업에 성공하자! 상담-능력개발-취업알선 3단계에 걸쳐 취업을 지...,만 18세 이상 ~ 34세 이하,"무관(단, 고교재학생은 3학년 2학기, 대학(원) 재학생은 마지막 학기 중 참여 가능)",무관,"무관(단, 재직자의 경우 근로시간이 주 30시간 미만 근로자는 참여 가능하고, 자영...",무관,-,상시,-
4,201903140006,내일배움카드(실업자ㆍ구직자),교육훈련·체험·인턴,구직자에게 직업능력개발훈련 기회를 제공하여 기술･기능 습득을 지원,만 15세 이상,-,-,미취업자,-,-,상시,-
5,201903140012,내일배움카드(근로자),교육훈련·체험·인턴,"중소기업 근로자, 비정규직 근로자 등의 직업훈련기회 확대를 통한 평생 고용가능성 제고",-,-,-,"중소기업 근로자, 기간제ㆍ단시간ㆍ파견ㆍ일용근로자, 이직 예정의 근로자(180일 이내...",-,-,"상시(지원기간: 2018. 7월부터, 예산 소진 시 조기마감 주의)",-
6,201903170001,청년 우대형 청약통장,생활비 지원 및 금융 혜택,"저소득, 무주택 청년을 위한 청약 통장",만 19세 이상~만 34세 이하의 청년,무관,무관,신고소득이 3000만 원 이하인 소득이 있는 청년,"저소득, 무주택 청년",-,2018년 7월 31일~2021년 12월 31일,-
7,201903140026,청년 보증부 월세대출,생활비 지원 및 금융 혜택,신혼부부·청년에게 월세대출 연 1%대의 저금리 상품 제공,만 34세 이하,무관,무관,연 소득 2천만원 이하 무주택 (예비)단독세대주,무관,-,상시,-
8,201903170002,중소기업취업청년 전월세보증금대출,주거 지원,중소기업에 취업한 청년들에게 저리로 중소기업 취업청년 전월세보증금을 지원,만 15세 이상 34세 이하인 청년,무관,무관,대출신청인과 배우자의 합산 총소득이 5천만원 이하인 청년,중소기업 취업자 또는 창업자,-,상시,-
9,201903140029,스마트 벤처 캠퍼스,R&D 지원,"유망지식서비스(앱/웹, 콘텐츠, SW, 융합)분야 청년창업자 집중 지원",만 39세 이하,무관,-,무관,-,-,사업 공고 시 신청,-


In [ ]:
youth_crawl_origin = """ CREATE TABLE  youth_crawl_origin(

                            id int; 
                            title varchar(100),
                            hash_tag varchar(20),
                            contents varchar(1000),
                            age varchar(200),
                            school varchar(200),
                            major varchar(200),
                            job_status varchar(200),
                            specific varchar(200),
                            company varchar(100),
                            apply_date varchar(1000),
                            else varchar(1000),
                            CONSTRAINT youth_origin_pk PRIMARY KEY (id)

);"""

In [23]:
df.iloc[1:2]

,id,title,hash tag,설명,연령,학력,전공,취업상태,특화분야,기업,신청기간,기타내용
1,201808230013,"월드프렌즈(World Friends Korea, WFK) - 일반봉사단",해외진출,개발도상국에 대한 주요 무상원조사업의 하나로 개발도상국의 수요를 바탕으로 우리가 비...,"만 19세 이상(단, 남자는 군필이거나 면제)",무관,무관,무관(기업체 등에 소속된 지원자는 소속기관과 협의하여 국내교육 입교 전까지 신분 정...,-,-,"연 8회 모집, 홈페이지 참조",-


In [ ]:
sqlalchemy.exc.IntegrityError